### Imports

In [ ]:
# Scrap 
import requests
import ScraperFC as sfs

# Data Managment 
import pandas as pd
import numpy as np

# Viz
from mplsoccer import Pitch, VerticalPitch
import matplotlib.pyplot as plt 

# Others
import os 
from PIL import Image
from io import BytesIO
import tweepy

In [ ]:
# Fonts and colors 
from matplotlib import font_manager
locations = ['/Users/emilio/Documents/3- Resources/lightstats/fonts/Montserrat']
font_files = font_manager.findSystemFonts(fontpaths=locations)
for file in font_files:
    font_manager.fontManager.addfont(file)
font = 'Montserrat'

# Define the colors using hexadecimal representations
background_color = '#F7F7F7' 
title_color = '#000000'   
text_color = '#333333'  
sub_text_color = '#808080'   
accent_color1 = '#4682B4'    
accent_color2 = '#ADD8E6' 

### Scrap info

In [ ]:
# Scrap sofascore
sofascore = sfs.Sofascore()

sofascore_url = 'https://www.sofascore.com/granada-real-madrid/EgbsEAn#id:11368634'
sofascore_url = sofascore_url.replace('id:', '')


# Scrap fotmob
fotmob_url = 'https://www.fotmob.com/api/matchDetails?matchId=4205687'

payload = {}
headers = {}

response = requests.request("GET", fotmob_url, headers=headers, data=payload)
data = response.json()

# print(response.text) Imprime el html

### DataFrames

In [ ]:
# DataFrames of shots and data stats
dfShots = pd.DataFrame(response.json()['content']['shotmap']['shots'])
df_matchStats = sofascore.get_general_match_stats(sofascore_url)

### Game Data

In [ ]:
# Translate the rounds to spanish 
round = data['general']['leagueRoundName']
round = round.replace('Round of 16', 'Octavos de Final')
round = round.replace('Round', 'Jornada')
round = round.replace('Semi-Finals', 'Semifinal')
round = round.replace('Quarter-Finals', 'Cuartos de Final')

match_time = data['general']['matchTimeUTC']
season = data['general']['parentLeagueSeason']
score = data['header']['status']['scoreStr']
pen_flag = data['header']['status']['reason']['short']
league = sofascore.get_match_data(sofascore_url)['tournament']['uniqueTournament']['name']

if pen_flag == 'Pen':
    away_penalties = data['header']['status']['reason']['penalties'][1]
    home_penalties = data['header']['status']['reason']['penalties'][0]

### Team's Data

In [ ]:
# Check if 'general' key exists in the response
home_name = data['header']['teams'][0]['name']
home_id = data['header']['teams'][0]['id']
home_color = dfShots[dfShots['teamId'] == home_id]['teamColor'].iloc[0]
home_img_url = data['header']['teams'][0]['imageUrl']

away_name = data['header']['teams'][1]['name']
away_id = data['header']['teams'][1]['id']
away_color = dfShots[dfShots['teamId'] == away_id]['teamColor'].iloc[0]
away_img_url = data['header']['teams'][1]['imageUrl']

In [ ]:
""" Get if the colors are similar """

def get_rgb(color):
    """Converts a hex color to an RGB tuple."""
    color = int(color[1:], 16)
    r = color >> 16
    g = (color - (r << 16)) >> 8
    b = color - (r << 16) - (g << 8)
    return [r, g, b]

def is_similar(color1, color2):
    """Checks if two colors are similar."""
    r1, g1, b1 = get_rgb(color1)
    r2, g2, b2 = get_rgb(color2)
    return (abs(r1 - r2) + abs(g1 - g2) + abs(b1 - b2)) < 50


if is_similar(home_color, away_color):
    home_color = '#FF0000'
    away_color = '#0000ff'

### Team's Stats

In [ ]:
# Extracting the 'stats' dictionary from the 'expected_goals_stats' dictionary
expected_goals_stats = data['content']['stats']['Periods']['All']['stats'][2]['stats'][1]
# Extracting the array of expected goals values from the 'expected_goals_stats' dictionary
expected_goals_array = expected_goals_stats['stats']

# List of all stats wanted
all_stats = ['xG', 'Possesion', 'Shots', 'Fouls', 'Big Chances']

# Home and Away data
home_data = {
    'xG': float(expected_goals_array[0]),
    'Possesion': float(df_matchStats[df_matchStats['name'] == 'Ball possession']['homeValue'].iloc[0]),
    'Shots': int(df_matchStats[df_matchStats['name'] == 'Total shots']['home'].iloc[0]),
    'Fouls': int(df_matchStats[df_matchStats['name'] == 'Fouls']['home'].iloc[0]),
    'Big Chances': float(df_matchStats[df_matchStats['name'] == 'Big chances']['home'].iloc[0])
}
away_data = {
    'xG': float(expected_goals_array[1]),
    'Possesion': float(df_matchStats[df_matchStats['name'] == 'Ball possession']['awayValue'].iloc[0]),
    'Shots': int(df_matchStats[df_matchStats['name'] == 'Total shots']['away'].iloc[0]),
    'Fouls': int(df_matchStats[df_matchStats['name'] == 'Fouls']['away'].iloc[0]),
    'Big Chances': float(df_matchStats[df_matchStats['name'] == 'Big chances']['away'].iloc[0])
}

### Shotmap

In [ ]:
# Filter Home and Away shots
home_shots = dfShots[dfShots['teamId'] == home_id]
away_shots = dfShots[dfShots['teamId'] == away_id]

# Home and Away goals
home_goal_shots = home_shots[home_shots['eventType'] == 'Goal']
away_goal_shots = away_shots[away_shots['eventType'] == 'Goal']

# Home and Away non-goals
home_non_goal_shots = home_shots[home_shots['eventType'] != 'Goal']
away_non_goal_shots = away_shots[away_shots['eventType'] != 'Goal']

### POM Stats

In [ ]:
# Fotmob POM DataFrame
dfPom = data['content']['matchFacts']['playerOfTheMatch']

# Extract the player of the match details
pom_stats = dfPom['stats']
pomFM = dfPom['name']['fullName']
pom_position = dfPom['role']
pom_id = dfPom['id']
pom_team = dfPom['teamName']
pom_img_url = f'https://images.fotmob.com/image_resources/playerimages/{pom_id}.png'

# Initialize empty lists to store the keys and values for plotting
keys = []
values = []

# Stats depending on positions 
positions = ['Keeper', 'Defender', 'Midfielder', 'Attacker']
attackerKeys = ['Goals', 'Assist', 'Total shots','Expected goals (xG)', 'Shot accuracy', 'Successful dribbles', 'xG Non-penalty']
midfielderKeys = ['Goals', 'Assists', 'Expected assists (xA)', 'Accurate passes', 'Chances created', 'Accurate long balls', 'Passes into final third', 'Successful dribbles']
defenderKeys = ['Tackles won', 'Blocks', 'Recoveries', 'Interceptions', 'Clearances']
keeperKeys = ['Clearances', 'Recoveries', 'Defensive actions']

# Define a dictionary to map positions to keys
position_keys_mapping = {
    'Keeper': keeperKeys,
    'Defender': defenderKeys,
    'Midfielder': midfielderKeys,
    'Attacker': attackerKeys
}

# Extracts and converts specific statistics from a dataset based on a given position.
if pom_position in positions:
    selected_keys = position_keys_mapping[pom_position]
    print("Selected keys:", selected_keys)  # Debugging statement

    # Iterate over the stats dictionary to extract the specified keys and values
    for stat_group in pom_stats:
        # Check if the title matches the desired titles
        if stat_group['title'] in ['Top stats', 'Attack', 'Defense']:
            # Extract the 'stats' dictionary
            stats_dict = stat_group['stats']
            
            # Iterate over the keys and store the values for specified keys
            for key in selected_keys:
                if key in stats_dict:
                    keys.append(key)
                    # Convert value to a numeric type
                    value = stats_dict[key]['stat']['value']
                    print("Value for", key, ":", value)  # Debugging statement
                    if isinstance(value, str) and '/' in value:  # Handling cases like '28/40 (70%)'
                        value = float(value.split('/')[0])  # Extracting the numerator as a float
                    else:
                        value = float(value)  # Convert to float if possible

                    values.append(value)

### Momentum

In [ ]:
# Match Momentum DataFrame 
match_momentum = sofascore.match_momentum(sofascore_url)

# Home and Away Match Momentum (df)
match_momentum_home = match_momentum[match_momentum['value'] > 0]
match_momentum_visit = match_momentum[match_momentum['value'] < 0]

### Dashboard

In [ ]:
""" Dashboard including the shots, match momentum, etc """
fig = plt.figure(figsize=(16, 9), constrained_layout=True)
gs = fig.add_gridspec(nrows=6, ncols=9)
fig.set_facecolor(background_color)

# Plotting Game Data
ax1 = fig.add_subplot(gs[0, 1:7])
if pen_flag == 'Pen':
    ax1.text(0.5, 0.70, f'{home_name} ({home_penalties}) {score} ({away_penalties}) {away_name}', weight='bold', fontsize=25, ha='center', fontfamily=font, color=title_color)
else:
    ax1.text(0.5, 0.70, f'{home_name} {score} {away_name}', weight='bold', fontsize=30, ha='center', fontfamily=font, color=title_color)
ax1.text(0.5, 0.25, f'{league}: {round}', fontsize=15, ha='center', fontfamily=font, color=sub_text_color)
ax1.axis(False)

# Fetch and plot the home team image
ax10 = fig.add_subplot(gs[0, :1])
response = requests.get(home_img_url)
img_data = Image.open(BytesIO(response.content))
ax10.imshow(img_data)
ax10.axis(False)

# Fetch and plot the away team image
ax11 = fig.add_subplot(gs[0, 7:8])
response = requests.get(away_img_url)
img_data = Image.open(BytesIO(response.content))
ax11.imshow(img_data)
ax11.axis(False)

# Plotting Game Stats
ax2 = fig.add_subplot(gs[1:3, :3])  
# Crear la figura y los subgráficos dentro de ax2
axs = [ax2]  # Utilizar una lista para axs para que coincida con la estructura existente
# Bucle sobre todas las estadísticas
for i, stat in enumerate(all_stats):
    # Calcular la suma total de la estadística
    total_stat = home_data[stat] + away_data[stat]
    
    # Calcular los porcentajes de contribución del equipo home y visitante
    porcentaje_home = home_data[stat] / total_stat
    porcentaje_visitante = away_data[stat] / total_stat
    
    # Crear la barra horizontal con dos colores
    ax2.barh(i, porcentaje_home, color=home_color, height=0.5)
    ax2.barh(i, porcentaje_visitante, color=away_color, left=porcentaje_home, height=0.5)
    
    # Añadir el valor numérico para el equipo home y visitante
    ax2.text(0.1, i, f'{home_data[stat]}', fontsize=14, ha='center', va='center', color='white', fontfamily=font)
    ax2.text(0.9, i, f'{away_data[stat]}', fontsize=14, ha='center', va='center', color='white', fontfamily=font)
    ax2.text(0.5, i, stat, fontsize=14, ha='center', va='center', color='white', fontfamily=font)
ax2.axis(False)

ax3 = fig.add_subplot(gs[1:3, 3:8])

# Plot the home and away match momentum
ax3.bar(match_momentum_home.minute, match_momentum_home.value, color=home_color)
ax3.bar(match_momentum_visit.minute, match_momentum_visit.value, color=away_color)

# Add text annotations for minutes 1, 46, 90, and 120 if they exist
lowest_value = min(match_momentum_home.value.min(), match_momentum_visit.value.min())
ax3.text(1, lowest_value - 5, '0\'', ha='right', va='center', weight='bold', fontfamily=font, color=text_color)
ax3.text(46, lowest_value - 5, 'HT', ha='center', va='center', weight='bold', fontfamily=font, color=text_color)
ax3.text(90, lowest_value - 5, "90'", ha='center', va='center', weight='bold', fontfamily=font, color=text_color)

# Check if the 120th minute exists and plot it if it does
if 120 in match_momentum_home.minute.values or 120 in match_momentum_visit.minute.values:
    ax3.text(120, lowest_value - 5, "120'", ha='center', va='center', weight='bold', fontfamily=font, color=text_color)

# Set the title and hide axis
ax3.set_title('Match Momentum', weight='bold')
ax3.axis(False)


ax4 = fig.add_subplot(gs[3:6, :3])
y_start = 0.6
# Plot the name and position
ax4.text(0.3, 0.8, f'{pom_name}', weight='bold', fontsize=20, fontfamily=font, color=text_color)
ax4.text(0.3, 0.75, f'{pom_position}, {pom_team}', fontsize=10, fontfamily=font, color=text_color)
# Iterate over keys and values to plot them horizontally
for i, (key, value) in enumerate(zip(keys, values)):
    # Calculate x-coordinate for the current key-value pair
    x_key = 0
    x_value = 0.6
    # Calculate y-coordinate for the current key-value pair
    y = y_start - i * 0.1
    # Make the key bold and plot it
    ax4.text(x_key, y, f'{key}:', fontsize=12, fontfamily=font, color=text_color)
    # Plot the value
    ax4.text(x_value, y, f'{value}', fontsize=12, weight= 'bold', fontfamily=font, color=text_color)
ax4.axis(False)

ax5 = fig.add_subplot(gs[3:4, :1])
response = requests.get(pom_img_url)
img_data = Image.open(BytesIO(response.content))
ax5.imshow(img_data)
ax5.axis(False)

ax6 = fig.add_subplot(gs[3:, 3:8])
ax6.text(26.25, 5,f'{home_name} shots', color=home_color, ha='center', fontsize=12, fontfamily=font)
ax6.text(78.75, 5,f'{away_name} shots', color=away_color, ha='center', fontsize=12, fontfamily=font)
pitch = Pitch(pitch_type='custom', pitch_length=105, pitch_width=68)
pitch.draw(ax=ax6)
if not home_goal_shots.empty:
    pitch.scatter(105-home_goal_shots.x, 68-home_goal_shots.y, ax=ax6, marker='football', s=200)
if not away_goal_shots.empty:
    pitch.scatter(away_goal_shots.x, away_goal_shots.y, ax=ax6, marker='football', s=200)
if not home_non_goal_shots.empty:
    pitch.scatter(105-home_non_goal_shots.x, 68-home_non_goal_shots.y, ax=ax6, alpha=0.6, label=home_name, color=home_color)
if not away_non_goal_shots.empty:
    pitch.scatter(away_non_goal_shots.x, away_non_goal_shots.y, ax=ax6, alpha=0.6, label=away_name, color=away_color)
#ax6.legend(loc='lower left')
ax6.axis(False)

fig.text(.63, 0.01, "Made by: @3zavalam     |    Data: SofaScore y FotMob", fontsize=10, ha='center', fontfamily=font, color=text_color)

# Replace "/" with "-" in the season string
season = season.replace("/", "-")
# Create the directory path
directory = os.path.join("output", "games", league, season, round)
# Check if the directory exists, if not, create it
if not os.path.exists(directory):
    os.makedirs(directory)
# Define the file path
file_path_game = os.path.join(directory, f"{home_name}_{round}_{season}_{away_name}.png")
plt.savefig(file_path_game)

plt.close()
print(f"Figure saved at: {file_path_game}")

### Twitter

In [ ]:
def tweet(file_path, text):
        auth = tweepy.OAuthHandler(api_key, api_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth, wait_on_rate_limit=True)

        # V2 Twitter API Authentication
        client = tweepy.Client(
            bearer_token,
            api_key,
            api_secret,
            access_token,
            access_token_secret,
            wait_on_rate_limit=True,
        )

        media_id = api.media_upload(filename=file_path).media_id_string
        client.create_tweet(text=text, media_ids=[media_id])
        print("Tweeted!")

In [ ]:
if 'Liga' in league or 'CONMEBOL' in league or 'CONCACAF' in league:
    tweet_text = (f"Estadisticas {home_name} vs {away_name}\n{league} - {round}")

else:
    tweet_text = (f"Statistics {home_name} vs {away_name}\n{league} - {round}")

In [ ]:
tweet(file_path=file_path_game, text=tweet_text)